This is a minimal code to see how to save a final model, plus additional information, that can be used later by Streamlit

In [ ]:
# This cell creates the final model, trained with the entire dataset (X, y)
# In a more complex case (like that of the assignment), the final model would be an entire pipeline (with preprocessing)
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from pathlib import Path
import pickle
# as_frame=True is for obtaining the data in dataframe format (instead of a numpy array).
# This keeps the data type for each of the columns (categorical, numerical)
# When we read a data file using pandas (as in the assignment), we are also reading a dataframe
COMPETITION_DATA_PATH = Path('dataset', 'bank_competition.pkl')
competition_data = pd.read_pickle(COMPETITION_DATA_PATH)
# X_df = iris.data
# y = iris.target
# from sklearn.tree import DecisionTreeClassifier
# final_model = DecisionTreeClassifier()
# final_model.fit(X_df, y)

NameError: name 'COMPETITION_DATA_PATH' is not defined

Now, let's create some additional information, such as for each variable, which are the possible values, so that it can be used later by Streamlit.

In [ ]:
# This cell is for the numerical features. In the Iris dataset, all of them are numerical
#   but in the assignment, some of them are categorical.
num_cols=X_df.select_dtypes(include=["number"]).columns.tolist()
num_summary = {}
for c in num_cols:
    s = X_df[c].dropna()
    num_summary[c] = {
        "p1": float(np.percentile(s, 1)),
        "median": float(np.percentile(s, 50)),
        "p99": float(np.percentile(s, 99))
    }
print (num_summary)

{'sepal length (cm)': {'p1': 4.4, 'median': 5.8, 'p99': 7.7}, 'sepal width (cm)': {'p1': 2.2, 'median': 3.0, 'p99': 4.150999999999999}, 'petal length (cm)': {'p1': 1.149, 'median': 4.35, 'p99': 6.7}, 'petal width (cm)': {'p1': 0.1, 'median': 1.3, 'p99': 2.5}}


In [ ]:
# This cell is for the non-numerical features (== categorical)
cat_cols=X_df.select_dtypes(exclude=["number"]).columns.tolist()
cat_options = {col: sorted(list(X_df[col].dropna().astype(str).str.strip().unique()))
               for col in cat_cols}
print (cat_options)

{}


In [ ]:
# Here, we pack and save into a joblib file, the final model, plus information about the numerical and categorical features
pack = {
    "model": final_model,
    "num_cols": num_cols,
    "cat_cols": cat_cols,
    "num_summary": num_summary,
    "cat_options": cat_options,
    "classes_": final_model.classes_.tolist(),
}

In [ ]:
from joblib import dump
dump(pack, "pack_for_streamlit.joblib")
print("if you have executed this code in Google Colab, you will need to download 'pack_for_streamlit' from google colab")

if you have executed this code in Google Colab, you will need to download 'pack_for_streamlit' from google colab
